In [1]:
import numpy as np
import os
import pandas as pd
import sys
sys.path.append('..')
from src.tools import numerai_api

In [2]:
round_number = numerai_api.get_current_round()

In [3]:
raw_data_path = os.path.join(os.path.pardir,'data','raw')
raw_data_file = os.path.join(raw_data_path, '{}_numerai_raw.pkl'.format(round_number))

In [4]:
!ls ../src/data/

get_raw_data_binary.py	get_raw_data.py  make_dataset.py


In [5]:
try:
    df = pd.read_pickle(raw_data_file)
except FileNotFoundError:
    !/home/sean/Projects/numerai/numerai/src/data/get_raw_data_binary.py
    df = pd.read_pickle(raw_data_file)


In [6]:
df.head()

,era,data_type,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
id,,,,,,,,,,,,,,,,,,,,,
n2b2e3dd163cb422,1.0,train,0.43487,0.44645,0.25802,0.37149,0.62235,0.67451,0.68103,0.45166,...,0.52962,0.42439,0.51680,0.46297,0.57426,0.57946,0.49646,0.48968,0.54194,1.0
n177021a571c94c8,1.0,train,0.50038,0.39216,0.38394,0.51213,0.36660,0.46911,0.68204,0.61320,...,0.51669,0.48445,0.57587,0.59860,0.67558,0.45577,0.80908,0.50287,0.61629,0.0
n7830fa4c0cd8466,1.0,train,0.47416,0.34143,0.39528,0.46337,0.72953,0.45962,0.47869,0.68118,...,0.41458,0.34804,0.29058,0.51382,0.36389,0.80602,0.39253,0.41821,0.58679,0.0
nc594a184cee941b,1.0,train,0.48759,0.55903,0.43987,0.38834,0.44650,0.46389,0.70749,0.63182,...,0.28776,0.42881,0.55402,0.53695,0.48793,0.62432,0.52898,0.49009,0.49557,0.0
nc5ab8667901946a,1.0,train,0.23433,0.55499,0.47849,0.56990,0.64945,0.47152,0.62085,0.57626,...,0.64405,0.32416,0.33193,0.58065,0.44587,0.47770,0.44020,0.47895,0.57978,0.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 637059 entries, n2b2e3dd163cb422 to n19eb23b343bd44e
Data columns (total 53 columns):
era          439975 non-null float64
data_type    637059 non-null object
feature1     637059 non-null float32
feature2     637059 non-null float32
feature3     637059 non-null float32
feature4     637059 non-null float32
feature5     637059 non-null float32
feature6     637059 non-null float32
feature7     637059 non-null float32
feature8     637059 non-null float32
feature9     637059 non-null float32
feature10    637059 non-null float32
feature11    637059 non-null float32
feature12    637059 non-null float32
feature13    637059 non-null float32
feature14    637059 non-null float32
feature15    637059 non-null float32
feature16    637059 non-null float32
feature17    637059 non-null float32
feature18    637059 non-null float32
feature19    637059 non-null float32
feature20    637059 non-null float32
feature21    637059 non-null float32
feature22    637059

In [8]:
df['data_type'].unique()

array(['train', 'validation', 'test', 'live'], dtype=object)

In [9]:
df_prediction = df.loc[(df['data_type'] == 'validation') | 
                       (df['data_type'] == 'test') | 
                       (df['data_type'] == 'live'), 'feature1':'feature50']

In [10]:
df_prediction.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
id,,,,,,,,,,,,,,,,,,,,,
nc698431c8a7849b,0.32517,0.43143,0.41502,0.38275,0.61453,0.54072,0.53334,0.61018,0.75716,0.41883,...,0.55700,0.49616,0.46299,0.34660,0.46200,0.62928,0.69258,0.57370,0.53505,0.62581
n8bbe152724604d3,0.21942,0.44113,0.54454,0.57169,0.55712,0.29693,0.44838,0.65719,0.58733,0.50112,...,0.59748,0.59925,0.24924,0.22513,0.67002,0.39358,0.60880,0.43593,0.55447,0.63251
ndd5cf7b9334c4e8,0.34701,0.44175,0.55301,0.29338,0.54576,0.55562,0.36476,0.66128,0.47317,0.32002,...,0.51613,0.66463,0.29351,0.39094,0.65684,0.49904,0.58673,0.25642,0.34658,0.53628
nf980f170d99e421,0.32640,0.40166,0.55554,0.48662,0.30937,0.33997,0.50130,0.80167,0.58813,0.62724,...,0.71046,0.42981,0.24457,0.17931,0.78263,0.29531,0.66034,0.53097,0.76424,0.56803
n35ca742811944cc,0.47259,0.58391,0.45705,0.23002,0.37732,0.47312,0.52077,0.54246,0.45543,0.44164,...,0.54333,0.41473,0.54475,0.53509,0.57826,0.65675,0.51514,0.62636,0.58010,0.34943


In [11]:
df_validation_predict = df.loc[df['data_type'] == 'validation','feature1':'feature50']
df_validation_target = df.loc[df['data_type'] == 'validation','target']

In [12]:
eras_validation = [era for era in df[df['data_type'] == 'validation'].era.unique()]
eras_validation

[121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0]

In [13]:
eras_train_counts = df.loc[df['data_type'] == 'train', 'era':'data_type'].groupby('era').count()
eras_train_counts.mean() * 12

data_type    39361.3
dtype: float64

In [14]:
X_train = df.loc[df['data_type'] == 'train', :].drop(['data_type','target'], axis=1)

In [15]:
X_train.head()

,era,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
id,,,,,,,,,,,,,,,,,,,,,
n2b2e3dd163cb422,1.0,0.43487,0.44645,0.25802,0.37149,0.62235,0.67451,0.68103,0.45166,0.82443,...,0.47780,0.52962,0.42439,0.51680,0.46297,0.57426,0.57946,0.49646,0.48968,0.54194
n177021a571c94c8,1.0,0.50038,0.39216,0.38394,0.51213,0.36660,0.46911,0.68204,0.61320,0.39951,...,0.56354,0.51669,0.48445,0.57587,0.59860,0.67558,0.45577,0.80908,0.50287,0.61629
n7830fa4c0cd8466,1.0,0.47416,0.34143,0.39528,0.46337,0.72953,0.45962,0.47869,0.68118,0.57239,...,0.57118,0.41458,0.34804,0.29058,0.51382,0.36389,0.80602,0.39253,0.41821,0.58679
nc594a184cee941b,1.0,0.48759,0.55903,0.43987,0.38834,0.44650,0.46389,0.70749,0.63182,0.36395,...,0.72995,0.28776,0.42881,0.55402,0.53695,0.48793,0.62432,0.52898,0.49009,0.49557
nc5ab8667901946a,1.0,0.23433,0.55499,0.47849,0.56990,0.64945,0.47152,0.62085,0.57626,0.50469,...,0.51905,0.64405,0.32416,0.33193,0.58065,0.44587,0.47770,0.44020,0.47895,0.57978


In [16]:
y_train = df.loc[df['data_type'] == 'train', ['era','target']]
y_train.head()

,era,target
id,,
n2b2e3dd163cb422,1.0,1.0
n177021a571c94c8,1.0,0.0
n7830fa4c0cd8466,1.0,0.0
nc594a184cee941b,1.0,0.0
nc5ab8667901946a,1.0,0.0


In [17]:
from sklearn.model_selection import GroupKFold

In [18]:
gkf = GroupKFold(n_splits=12)
gkf_split = gkf.split(X_train, y_train, groups=X_train['era'])

X_split = []
y_split = []

for split in gkf_split:
    X_split.append(split[0])
    y_split.append(split[1])
    
splits = zip(X_split, y_split)

In [19]:
splits = [(split1, split2) for split1, split2 in splits]

In [20]:
for split_x, split_y in splits:
    print(len(split_y)/len(split_x))

0.09186509697749767
0.09118404528707783
0.09213775575323385
0.09116287063937371
0.08977922732331818
0.09118707030383677
0.08982750505302212
0.09178938258797685
0.0895590144466989
0.09150479042746419
0.09019576344419579
0.09072443768306301


In [21]:
X_split_train = []
y_split_train = []
X_split_test = []
y_split_test = []

gkf = GroupKFold(n_splits=10)
for train, test in gkf.split(X_train, y_train, groups=X_train['era']):
    X_split_train.append(X_train.iloc[train])
    y_split_train.append(y_train.iloc[train])
    X_split_test.append(X_train.iloc[test])
    y_split_test.append(y_train.iloc[test])

In [22]:
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression()

The following is done with cross_val_score

for i in range(0,10):
    
    log_reg_model.fit(X_split_train[i].drop('era', axis=1), y_split_train[i]['target'])
    score = log_reg_model.score(X_split_test[i].drop('era', axis=1), y_split_test[i]['target'])
    print(score)

In [23]:
from sklearn.model_selection import cross_val_score
cv = gkf.split(X_train, y_train, groups=X_train['era'])
#cv_score = cross_val_score(log_reg_model, X_train.drop('era', axis=1), y_train['target'], cv=cv, scoring='neg_log_loss') 

In [24]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

lg = LogisticRegression()
cv = gkf.split(X_train, y_train, groups=X_train['era'])
#cv_log_loss = cross_val_score(lg, X_train.drop('era', axis=1), y_train['target'], cv=cv, scoring='neg_log_loss') 

In [25]:
def create_submission(df_predict_feat, model, filename='predictions.csv'):
    submission = pd.DataFrame()
    submission['id'] = df_predict_feat.index
    submission['probability'] = model.predict_proba(df_predict_feat)[:,1]
    submission.to_csv(filename,index=None)

In [26]:
import logging
# setup logger
log_fmt = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_fmt)
logger = logging.getLogger()
logger.handlers[0].stream = sys.stdout

In [27]:
lg_baseline = LogisticRegression()
logger.info('Fitting model')
lg_baseline.fit(X_train.drop('era', axis=1),y_train['target'])
logger.info('Model fit')
logger.info('Creating submission')
create_submission(df_prediction, lg_baseline, 'predictions_baseline.csv')
logger.info('Submission created')
"""
submission = pd.DataFrame()
submission['id'] = df_prediction.index
submission['probability'] = lg_baseline.predict_proba(df_prediction)[:,1]
submission.to_csv('predictions_baseline.csv',index=None)
"""


2018-01-28 11:03:01,172 - INFO - Fitting model
2018-01-28 11:03:17,190 - INFO - Model fit
2018-01-28 11:03:17,191 - INFO - Creating submission
2018-01-28 11:03:17,849 - INFO - Submission created


"\nsubmission = pd.DataFrame()\nsubmission['id'] = df_prediction.index\nsubmission['probability'] = lg_baseline.predict_proba(df_prediction)[:,1]\nsubmission.to_csv('predictions_baseline.csv',index=None)\n"

In [28]:
def get_validation_log_loss(model, df):
    df_validation_predict = df.loc[df['data_type'] == 'validation','feature1':'feature50']
    df_validation_target = df.loc[df['data_type'] == 'validation','target']
    validation_prediction = model.predict_proba(df_validation_predict)
    return log_loss(df_validation_target, validation_prediction)
    

In [29]:
baseline_logloss = get_validation_log_loss(lg_baseline, df)
baseline_logloss

0.6929459596319582

In [30]:
cv = gkf.split(X_train, y_train, groups=X_train['era'])
lg_group = LogisticRegressionCV(cv=cv)
logger.info('Starting model fit')
lg_group.fit(X_train.drop('era', axis=1), y_train['target'])
logger.info('Model fitted')

2018-01-28 11:03:17,990 - INFO - Starting model fit
2018-01-28 11:06:18,937 - INFO - Model fitted


In [31]:
group_logloss = get_validation_log_loss(lg_group, df)
group_logloss

0.6925454807645468

In [32]:
(group_logloss-baseline_logloss)/group_logloss*100

-0.05782708551779613

In [33]:
create_submission(df_prediction, lg_group, 'predictions_groupkfold.csv')

In [39]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=500,max_leaf_nodes=15)
logger.info('Fitting model')
rfc.fit(X_train.drop('era', axis=1),y_train['target'])
logger.info('Model fit')
rfc_logloss = get_validation_log_loss(rfc, df)
rfc_logloss

2018-01-28 19:47:29,729 - INFO - Fitting model
2018-01-28 19:56:49,136 - INFO - Model fit


0.6926281272932799

0.7809569239067203

0.7809569239067203 with default
0.6926539978867465 with n_estimators=1000,max_leaf_nodes=15
0.6926281272932799


In [43]:
from sklearn.model_selection import GridSearchCV
cv = gkf.split(X_train, y_train, groups=X_train['era'])
param_grid = { 
    'n_estimators': [50, 100, 200],
    'max_features': [10,20,30,40,50],
    'max_leaf_nodes': [10,15,20,25,50]
}
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=500,max_leaf_nodes=15)
CV_rfc = GridSearchCV(n_jobs=8, estimator=rfc, param_grid=param_grid, cv=cv)
logger.info('Starting model fit')
CV_rfc.fit(X_train.drop('era', axis=1), y_train['target'])
logger.info('Model fitted')
print (CV_rfc.best_params_)

2018-01-28 21:18:10,173 - INFO - Starting model fit


KeyboardInterrupt: 